In [ ]:
#pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

In [41]:
from datasets import load_dataset

csv_file = "./results/qa.csv"
# instruct_tune_dataset = load_dataset('csv', data_files=csv_file, split="train", cache_dir='data_cache')
instruct_tune_dataset = load_dataset('csv', data_files=csv_file, cache_dir='data_cache')

In [42]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 420
    })
})

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16,
)

In [4]:
MODEL_PATH = f"../models/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=nf4_config,
    device_map='auto',
    local_files_only=True,
    #trust_remote_code=False,
    use_cache=False
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [26]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')
  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, 
                                 do_sample=True, 
                                 pad_token_id=tokenizer.pad_token_id, # pad_token_id=tokenizer.eos_token_id
                                 eos_token_id=tokenizer.eos_token_id)
  decoded_output = tokenizer.batch_decode(generated_ids)
  return decoded_output[0].replace(prompt, "")


In [18]:
prompt="### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.\n\n### Response:"
prompt="<s> [INST]\nWhat is your namn? [/INST]"
prompt="### Instruction:\n{instruction}### Input:\n{user_input}\n\n### Response:"
prompt="[INST]{user_input}[/INST]"
prompt = prompt.format(instruction="", user_input="what is your name?")

In [27]:
generate_response(prompt, model)

'<s>  My name is Mistral 7B v0.1. But you can just call me Mistral.</s>'

In [34]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj','lm_head']
)

In [35]:
model2 = prepare_model_for_kbit_training(model)
model2 = get_peft_model(model2, peft_config)

In [56]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "./results/mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 2, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 2,
  #gradient_accumulation_steps=2,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=1e-4,
  bf16=True,
  lr_scheduler_type='constant',
  #report_to="tensorboard"
)

In [ ]:
def create_prompt(sample):
  bos_token = "<s>"
  original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  input = sample["response"]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += "### Instruction:"
  full_prompt += "\n" + system_message
  full_prompt += "\n\n### Input:"
  full_prompt += "\n" + input
  full_prompt += "\n\n### Response:"
  full_prompt += "\n" + response
  full_prompt += eos_token

  return full_prompt

In [44]:
def create_prompt2(sample):
   return sample["text"]

In [52]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt2, # this will aplly the create_prompt mapping to all training and test dataset
  args=args,
  train_dataset=instruct_tune_dataset["train"],
  #eval_dataset=instruct_tune_dataset["test"]
)

/home/flash/miniconda3/envs/torch/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:234: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [53]:
trainer.train(resume_from_checkpoint=True)

/home/flash/miniconda3/envs/torch/lib/python3.10/site-packages/trl/trainer/utils.py:570: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


Step,Training Loss,Validation Loss


TrainOutput(global_step=3, training_loss=0.6304134925206503, metrics={'train_runtime': 101.9456, 'train_samples_per_second': 0.039, 'train_steps_per_second': 0.02, 'total_flos': 536794575667200.0, 'train_loss': 0.6304134925206503, 'epoch': 0.01})

In [54]:
trainer.save_model("./results/mistral_instruct_generation")